<a href="https://colab.research.google.com/github/iwatake2222/study_karaage_ai/blob/master/karaage_ai_02_01_webcamera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp drive/MyDrive/my_model.h5 .

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

2.5.0


In [ ]:
keras_model = tf.keras.models.load_model("my_model.h5")
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_5 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 61504)            

In [ ]:
labels = []
with open("./labels.txt", "r") as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)
NUM_CLASSES = len(labels)

['choki', 'gu', 'pa']


In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py

In [ ]:
import colab_camera

In [ ]:
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def run(img_str):
  decimg = base64.b64decode(img_str.split(",")[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8)
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  out_img = decimg
  cv2.putText(out_img, labels[result], (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

  _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8'))  
  return IPython.display.JSON({"img_str": img_str})

output.register_callback("notebook.run", run)
                     

In [ ]:
colab_camera.use_cam()